In [14]:
from bs4 import BeautifulSoup
import requests, yaml, pytz
import datetime as dt
import pandas as pd

In [15]:
efc_url = "https://www.efinancialcareers.co.uk/jobs/in-United-Kingdom"

url = efc_url

In [16]:
def parser():
    headers = headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
    content = requests.get(url, headers).content
    soup = BeautifulSoup(content, "html.parser")

    return soup

In [17]:
def job_card():
    soup = parser()
    
    efc_jobcard = soup.find('efc-job-search-results').find_all('div', {'class': 'card-info'})

    #job_component = [card.find('div', {'class': 'card-info'}) for card in efc_jobcard]
    #card_details = [card.find('efc-card-details') for card in efc_jobcard]
    #card_footer = [card.find('efc-card-footer') for card in efc_jobcard]
        
    return efc_jobcard

In [18]:
def first_page():
    jobcard = job_card()

    job_title, company, job_link, location, position_type, work_arrangement, salary = [], [], [], [], [], [], []
    job_dct = {}

    for item in jobcard:
        job_title.append(item.find('h3').text)
        company.append(item.find('div', {'class': 'font-body-3'}).text.lstrip().rstrip())
        job_link.append(item.find('a').get('href'))
        location.append(item.find('span', {'class': 'dot-divider'}).text)
        position_type.append(item.find('span', {'class': 'dot-divider ng-star-inserted'}).text)

        try: work_arrangement.append(item.find('span', {'class': 'dot-divider-after ng-star-inserted'}).text)
        except: work_arrangement.append(None)

        try: salary.append(item.find('span', {'class': 'dot-divider-after last-job-criteria ng-star-inserted'}).text)
        except: salary.append(None)

    job_dct = {"Position": job_title, "Company": company, "Link": job_link, "Location": location, "Position Type": position_type,
               "Work Arrangement": work_arrangement, "Salary": salary}
    final_df = pd.DataFrame(job_dct)
    
    return final_df

In [19]:
test = first_page()
print(test)



                                             Position          Company  \
0                                Broker Sales Manager   Funding Circle   
1                           HR Operations Coordinator   Houlihan Lokey   
2                           LIBOR Discontinuation AVP              LMA   
3                   Senior Manager Indirect Tax / VAT    Tandem Search   
4   Customer Journey Manager/Product Owner - Retai...      Harvey Nash   
5   Finance Systems Analyst: SAP Central Finance (...           Aramco   
6   Financial Data Scientist: AI (Saudi Arabia based)           Aramco   
7          Group Finance Analyst (Saudi Arabia based)           Aramco   
8                    M&A Analyst (Saudi Arabia based)           Aramco   
9            M&A Finance Analyst (Saudi Arabia based)           Aramco   
10              Compliance Associate, Capital Markets           Bovill   
11                                     Client Manager  Legal & General   
12                   Business Analyst/